In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [2]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
# ...these are taken from predictions for the Run-5 CMS
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")

In [3]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [4]:
run5_system.triggers

,Name,Output,Data (bytes),Reduction Ratio,Skill mean,Skill variance,Link Efficiency (J/bit),Op Efficiency (J/op),Compression
0,Tracking,Intermediate,0,1.0,0,0,2.500000e-11,0.000,0
1,Timing,Intermediate,0,1.0,0,0,2.500000e-11,0.000,0
2,Calorimetry,Intermediate,0,1.0,0,0,2.500000e-11,0.000,0
3,Muon,Intermediate,0,1.0,0,0,2.500000e-11,0.000,0
4,Intermediate,Global,260000,53.3,3,1,2.500000e-11,0.003,0
5,Global,Disk,0,100.0,4,1,2.500000e-11,16.000,0
6,Disk,NaN,0,1.0,0,0,2.500000e-11,0.000,0


In [5]:
#calculate the total reduction to keep this constant across experiments
overall_reduction = run5_system.triggers.iloc[4]["Reduction Ratio"] * run5_system.triggers.iloc[5]["Reduction Ratio"]

In [6]:
overall_reduction

5330.0

In [7]:
40e6 / overall_reduction

7504.6904315197

In [8]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [9]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [10]:
"""
Vary the accept rate of the level 1 trigger and inspect its impact on performance and resources required
"""
def vary_system(system: System, functions, reduction: float, skill: float):
    t = system.triggers.copy()
    #intermediate reduction stage
    t.at[4, "Reduction Ratio"] = reduction
    #global reduction stage
    t.at[5, "Reduction Ratio"] = overall_reduction / reduction
    #change L1T skill
    t.at[4, "Skill mean"] = skill
    g = construct_graph(system.detectors, t, system.globals, functions)

    power = g.graph["op power"] + g.graph["link power"]
    confusion = g.graph["performance"]

    return confusion, power, g

In [12]:
#current l1t accept / skill
ex = vary_system(run5_system, funcs, 400, 3)

In [13]:
run5_graph = ex[2]

In [14]:
ex2 = vary_system(run5_system, funcs, 400, 5)[2]

In [15]:
ex2.nodes["Intermediate"]["classifier"].skill

5

In [16]:
#its predicted confusion matrix for the workflow
ex[0]

array([[3.99892130e+07, 3.28230313e+03],
       [3.28230292e+03, 4.22238730e+03]])

In [17]:
#and total required power (MW) using 2024's technology
ex[1] / 1e6

45.23322453243844

In [35]:
#vary this accept rate from today's rate to the planned Run-5 
l1t_reductions = np.linspace(400, 53.3, 101)
l1t_skills = np.linspace(3, 4, 101)

In [36]:
res = [[vary_system(run5_system, funcs, r, s) for r in l1t_reductions] for s in l1t_skills]

In [37]:
def extract_metrics(results):
    all_confusion = np.stack([r[0] for r in results])

    all_power = [r[1] / density_scale_model(r[2].graph["globals"]["Year"][0]) for r in results]
    all_power = np.array(all_power)

    all_recall = np.array([recall(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_f1 = np.array([f1_score(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    productivity = np.array([np.sum(get_passed(all_confusion[i,:,:])) for i in range(all_confusion.shape[0])])

    metrics = {"confusion": all_confusion,
               "power": all_power,
               "recall": all_recall,
               "f1 score": all_f1,
               "productivity": all_recall * productivity}

    return metrics

In [38]:
run5_metrics = [extract_metrics(r) for r in res]

In [39]:
f1 = np.stack([r["f1 score"] for r in run5_metrics])

In [54]:
power = np.stack([r["power"] for r in run5_metrics])

In [80]:
fig = go.Figure(data =
    go.Contour(
        z=f1,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "L1T Skill",
                  title = "Recall by L1T Skill & Reduction Ratio")
fig.update_xaxes(autorange="reversed")
fig.show()

In [82]:
fig = go.Figure(data =
    go.Contour(
        z=f1,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  title = "Recall by Trigger Skill & Reduction Ratio",
                  xaxis=dict(
                        title="Reduction Ratio",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the x-axis title
                    ),
                    yaxis=dict(
                        title="Skill",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the y-axis title
                    ),
                    font = dict(size=18,),)
fig.update_xaxes(autorange="reversed")
fig.show()

In [ ]:
#because its rejection is so much higher, there's more potential improvement gained by making L1T's skill higher 
#than simply passing more data to the HLT

## next - calibrate the level of the current L1T vs. HLT skill

In [69]:
fig = go.Figure(data =
    go.Scatter(
        y=power[1],
        x=l1t_reductions, # horizontal axis
    ))
fig.update_layout(width = 800, 
                  height = 600,
                  title = "Power by L1T Reduction",
                  xaxis_title = "L1T Reduction",
                  yaxis_title = "System Power (W)")
fig.update_xaxes(autorange="reversed")
fig.show()

In [72]:
fig = go.Figure(data =
    go.Contour(
        z = f1 / power * (40e6),
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800,
                  height = 600,
                  title = "Productivity (F1/Joule) by L1T Reduction & Skill",
                  xaxis_title = "L1T Reduction",
                  yaxis_title = "L1T Skill")
fig.update_xaxes(autorange="reversed")
fig.show()